In [85]:
import numpy as np
import pandas as pd
import re

In [86]:
env_data = pd.read_csv("data/env_data.csv", index_col=0)
env_data.head()

,Environment,Instance,vcpu,cpu_mem (GiB),cpu_type,gpu,gpu_mem (GiB),gpu_type
0,env1,c7a.medium,1,2.0,AMD EPYC Gen4,0,NaN,NaN
1,env2,c7a.large,2,4.0,AMD EPYC Gen4,0,NaN,NaN
2,env3,c7a.xlarge,4,8.0,AMD EPYC Gen4,0,NaN,NaN
3,env4,c7a.2xlarge,8,16.0,AMD EPYC Gen4,0,NaN,NaN
4,env5,c6i.xlarge,4,8.0,Intel Xeon 8375C (Ice Lake),0,NaN,NaN


## Data Cleaning (Runtime Data)
We will convert everything into milliseconds

In [87]:
runtime_data = pd.read_csv("data/runtime_data.csv", index_col=0)
runtime_data.head()

,Environment,trial,c_CNN_time,c_DNN_time,c_LSTM_time,c_T_time,g_CNN_time,g_DNN_time,g_LSTM_time,g_T_time
0,env1,1,71.589ms,60.689ms,221.471ms,357.480ms,NaN,NaN,NaN,NaN
1,env1,2,13.843ms,60.252ms,148.735ms,344.164ms,NaN,NaN,NaN,NaN
2,env1,3,13.284ms,59.453ms,143.268ms,360.393ms,NaN,NaN,NaN,NaN
3,env2,1,21.971ms,61.970ms,107.128ms,205.548ms,NaN,NaN,NaN,NaN
4,env2,2,11.617ms,60.317ms,105.891ms,200.521ms,NaN,NaN,NaN,NaN


In [90]:
# Convert a messy string format of runtime to a float or np.nan
def standardize_time(time):
    if (type(time)==str):
        time = time.strip()
        
        t = re.split(r'(\d+\.\d+)', time)
        t = [_ for _ in t if _ != '']
        
        if (t[1] == "ms"):
            return float(t[0])
        elif (t[1] == "s"):
            return float(t[0])*1000
        else:
            return np.nan
    else:
        return time
    
for col in runtime_data.columns[2:]:
    runtime_data[col] = runtime_data[col].apply(standardize_time).astype(float)

In [91]:
runtime_data.head()

,Environment,trial,c_CNN_time,c_DNN_time,c_LSTM_time,c_T_time,g_CNN_time,g_DNN_time,g_LSTM_time,g_T_time
0,env1,1,71.589,60.689,221.471,357.480,NaN,NaN,NaN,NaN
1,env1,2,13.843,60.252,148.735,344.164,NaN,NaN,NaN,NaN
2,env1,3,13.284,59.453,143.268,360.393,NaN,NaN,NaN,NaN
3,env2,1,21.971,61.970,107.128,205.548,NaN,NaN,NaN,NaN
4,env2,2,11.617,60.317,105.891,200.521,NaN,NaN,NaN,NaN


## Data Analysis (Runtime Data)